In [10]:
!pip install nba-on-court


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [11]:
from nba_api.stats.endpoints import playbyplayv2
import nba_on_court.nba_on_court as noc

In [25]:
pbp = playbyplayv2.PlayByPlayV2(game_id="0022100001").play_by_play.get_data_frame()
pbp_with_players = noc.players_on_court(pbp)
len(pbp_with_players.columns) - len(pbp.columns)
pbp_with_players.columns

Index(['GAME_ID', 'EVENTNUM', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'PERIOD',
       'WCTIMESTRING', 'PCTIMESTRING', 'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION',
       'VISITORDESCRIPTION', 'SCORE', 'SCOREMARGIN', 'PERSON1TYPE',
       'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_CITY',
       'PLAYER1_TEAM_NICKNAME', 'PLAYER1_TEAM_ABBREVIATION', 'PERSON2TYPE',
       'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_CITY',
       'PLAYER2_TEAM_NICKNAME', 'PLAYER2_TEAM_ABBREVIATION', 'PERSON3TYPE',
       'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY',
       'PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION',
       'VIDEO_AVAILABLE_FLAG', 'AWAY_PLAYER1', 'AWAY_PLAYER2', 'AWAY_PLAYER3',
       'AWAY_PLAYER4', 'AWAY_PLAYER5', 'HOME_PLAYER1', 'HOME_PLAYER2',
       'HOME_PLAYER3', 'HOME_PLAYER4', 'HOME_PLAYER5'],
      dtype='object')

In [28]:
pbp_with_players.iloc[0, 34:].reset_index(drop=True)


0     201142
1    1629651
2     201933
3     201935
4     203925
5     201572
6     201950
7    1628960
8     203114
9     203507
Name: 0, dtype: object

In [13]:
players_id = list(pbp_with_players.iloc[0, 34:].reset_index(drop=True))
print(players_id)

[201142, 1629651, 201933, 201935, 203925, 201572, 201950, 1628960, 203114, 203507]


In [14]:
players_name = noc.players_name(players_id)
print(players_name)

['Kevin Durant', 'Nic Claxton', 'Blake Griffin', 'James Harden', 'Joe Harris', 'Brook Lopez', 'Jrue Holiday', 'Grayson Allen', 'Khris Middleton', 'Giannis Antetokounmpo']


OSError: [Errno 30] Read-only file system: '/nbastats_2022.tar.xz'